In [16]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC


driver = webdriver.Chrome()
driver.get("https://www.opinet.co.kr/user/main/mainView.do")
wait = WebDriverWait(driver, 10)

# (1) hover 대상: "싼 주유소 찾기"
hover_element = wait.until(EC.presence_of_element_located(
    (By.CSS_SELECTOR, "#header > div > ul > li:nth-child(1) > a")
))

# (2) hover 동작 수행
ActionChains(driver).move_to_element(hover_element).perform()

# (3) hover 후 서브 메뉴 "경로별" 나타나므로 기다렸다가 클릭
menu_option = wait.until(EC.element_to_be_clickable(
    (By.CSS_SELECTOR, "#header > div > ul > li:nth-child(1) > ul > li:nth-child(1) > a > span")
))
menu_option.click()

# 시/도 선택
city = wait.until(EC.presence_of_element_located((By.ID, "SIDO_NM0")))
select_city = Select(city)
select_city.select_by_visible_text("부산")
time.sleep(1)

for i in range(0,16):
    # 구/군 선택
    gu = wait.until(EC.presence_of_element_located((By.ID, "SIGUNGU_NM0")))
    select_gu = Select(gu)
    time.sleep(1)
    
    # 구 목록 출력 및 선택
    gu_list = [option.get_attribute("value") for option in gu.find_elements(By.TAG_NAME, "option") if option.get_attribute("value")]
    select_gu.select_by_visible_text(gu_list[i])  #구 선택
    time.sleep(1)

    excel = wait.until(EC.element_to_be_clickable(
        (By.CSS_SELECTOR, "#templ_list0 > div:nth-child(7) > div > a > span")
    ))
    excel.click()


driver.quit()

In [30]:
# !pip install openpyxl

In [3]:
from io import BytesIO
from hdfs import InsecureClient
import pandas as pd

# HDFS 연결
hdfs = InsecureClient("http://namenode:9870", user="hadoop")

# 파일 경로 리스트
paths = [f"지역_위치별(주유소) ({i}).xls" for i in range(1, 16)]
paths.insert(0, "지역_위치별(주유소).xls")

# 파일 읽고 리스트에 저장
datas = []
for path in paths:
    with hdfs.read(path) as reader:
        bytes_data = reader.read()
        bio = BytesIO(bytes_data)
        df = pd.read_excel(bio)
        datas.append(df)

# 병합
df = pd.concat(datas, ignore_index=True)



--- DataFrame 0 ---
  지역_위치별(주유소)        Unnamed: 1             Unnamed: 2 Unnamed: 3  \
0         NaN               NaN                    NaN        NaN   
1          지역                상호                     주소         상표   
2       부산광역시             해안주유소        부산 강서구 낙동남로 432      SK에너지   
3       부산광역시  신창에너지㈜직영 송정셀프주유소  부산 강서구 낙동남로 127 (송정동)      S-OIL   
4       부산광역시             서강주유소   부산 강서구 낙동북로 42 (강동동)      SK에너지   

      Unnamed: 4 Unnamed: 5 Unnamed: 6 Unnamed: 7 Unnamed: 8   Unnamed: 9  
0            NaN        NaN        NaN        NaN        NaN  (단위 : 원/리터)  
1           전화번호       셀프여부      고급휘발유        휘발유         경유         실내등유  
2  070-8894-3569          Y          -       1549       1399            -  
3   051-972-5104          Y          -       1549       1409            -  
4   051-971-8484          Y          -       1557       1417            -  

--- DataFrame 1 ---
  지역_위치별(주유소)       Unnamed: 1              Unnamed: 2 Unnamed: 3  \
0         NaN     

In [31]:
import pandas as pd
import os
from glob import glob

folder_path = r"Downloads" 
all_files = glob(os.path.join(folder_path, "지역_위치별(주유소)*.xls"))

df_list = []

for file in all_files:
    try:
        df = pd.read_excel(file, skiprows=1)

        df = df[df.iloc[:, 0].notna()]  # 첫 번째 열 비어있지 않은 행만
        df = df[~df.iloc[:, 0].astype(str).str.contains("총")]

        df_list.append(df)

    except Exception as e:
        print(f"{file} 처리 중 오류: {e}")

combined_df = pd.concat(df_list, ignore_index=True)

print(combined_df.head())
print(f"총 {len(combined_df)}개 행으로 통합 완료")

combined_df.to_excel("부산_전체_주유소_통합.xlsx", index=False)


  Unnamed: 0       Unnamed: 1              Unnamed: 2 Unnamed: 3  \
0         지역               상호                      주소         상표   
1      부산광역시  한길주유소 금사점 한솔유화㈜    부산 금정구 반송로 420 (금사동)   HD현대오일뱅크   
2      부산광역시         훼미리알뜰주유소            부산 금정구 두실로 2      알뜰주유소   
3      부산광역시       은마석유 노포주유소  부산 금정구 중앙대로 2191 (노포동)      GS칼텍스   
4      부산광역시          청룡명품주유소        부산 금정구 중앙대로 2130   HD현대오일뱅크   

     Unnamed: 4 Unnamed: 5 Unnamed: 6 Unnamed: 7 Unnamed: 8 (단위 : 원/리터)  
0          전화번호       셀프여부      고급휘발유        휘발유         경유        실내등유  
1  051-524-6642          Y       1897       1549       1419           -  
2  051-582-5882          Y          -       1575       1437           -  
3  051-508-1645          Y       1849       1575       1437        1350  
4  051-508-5150          Y          -       1575       1437           -  
총 363개 행으로 통합 완료


In [33]:
combined_df.columns = ['local','name','addr','mark','tel','self','oil1','oil2','oil3','oil4']
combined_df

,local,name,addr,mark,tel,self,oil1,oil2,oil3,oil4
0,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
1,부산광역시,한길주유소 금사점 한솔유화㈜,부산 금정구 반송로 420 (금사동),HD현대오일뱅크,051-524-6642,Y,1897,1549,1419,-
2,부산광역시,훼미리알뜰주유소,부산 금정구 두실로 2,알뜰주유소,051-582-5882,Y,-,1575,1437,-
3,부산광역시,은마석유 노포주유소,부산 금정구 중앙대로 2191 (노포동),GS칼텍스,051-508-1645,Y,1849,1575,1437,1350
4,부산광역시,청룡명품주유소,부산 금정구 중앙대로 2130,HD현대오일뱅크,051-508-5150,Y,-,1575,1437,-
...,...,...,...,...,...,...,...,...,...,...
358,부산광역시,(주)서호주유소,부산 강서구 명지국제2로28번길 36,S-OIL,051-206-6661,Y,1998,1678,1518,-
359,부산광역시,만선주유소,부산 강서구 가락대로210번길 102,HD현대오일뱅크,051-831-9171,N,-,1698,1558,-
360,부산광역시,비케이원(주) 미래드림주유소,부산 강서구 가락대로 649,HD현대오일뱅크,051-974-0808,Y,-,1709,1559,-
361,부산광역시,대양산업(주)대양가덕주유소,부산 강서구 거가대로 2571,GS칼텍스,051-715-2201,Y,-,1788,1638,1400


In [37]:
combined_df = combined_df.drop(0).reset_index(drop=True)
combined_df

,local,name,addr,mark,tel,self,oil1,oil2,oil3,oil4,구군
0,부산광역시,한길주유소 금사점 한솔유화㈜,부산 금정구 반송로 420 (금사동),HD현대오일뱅크,051-524-6642,Y,1897,1549,1419,-,금정구
1,부산광역시,훼미리알뜰주유소,부산 금정구 두실로 2,알뜰주유소,051-582-5882,Y,-,1575,1437,-,금정구
2,부산광역시,은마석유 노포주유소,부산 금정구 중앙대로 2191 (노포동),GS칼텍스,051-508-1645,Y,1849,1575,1437,1350,금정구
3,부산광역시,청룡명품주유소,부산 금정구 중앙대로 2130,HD현대오일뱅크,051-508-5150,Y,-,1575,1437,-,금정구
4,부산광역시,금두꺼비주유소,부산광역시 금정구 부곡로 67 (부곡동),알뜰주유소,051-583-6433,Y,-,1587,1457,-,금정구
...,...,...,...,...,...,...,...,...,...,...,...
357,부산광역시,(주)서호주유소,부산 강서구 명지국제2로28번길 36,S-OIL,051-206-6661,Y,1998,1678,1518,-,강서구
358,부산광역시,만선주유소,부산 강서구 가락대로210번길 102,HD현대오일뱅크,051-831-9171,N,-,1698,1558,-,강서구
359,부산광역시,비케이원(주) 미래드림주유소,부산 강서구 가락대로 649,HD현대오일뱅크,051-974-0808,Y,-,1709,1559,-,강서구
360,부산광역시,대양산업(주)대양가덕주유소,부산 강서구 거가대로 2571,GS칼텍스,051-715-2201,Y,-,1788,1638,1400,강서구


In [38]:
df1 = pd.read_csv('부산_전체_주유소_수정.csv')
c_data = df1.sort_values('휘발유', ascending=True).head(5)
e_data = df1.sort_values('휘발유', ascending=False).head(5)
df1

,구,주소,상호,휘발유,경유,위도,경도
0,강서구,부산 강서구 낙동남로 432,해안주유소,1549.0,1399.0,35.111787,128.867467
1,강서구,부산 강서구 낙동남로 127 (송정동),신창에너지㈜직영 송정셀프주유소,1549.0,1409.0,35.101886,128.836964
2,강서구,부산 강서구 낙동북로 364,은마석유 경기장주유소,1557.0,1417.0,35.211911,128.968054
3,강서구,부산 강서구 낙동북로 209 (대저1동),동방석유㈜직영 대저주유소,1557.0,1417.0,35.214206,128.951693
4,강서구,부산 강서구 낙동북로 142,우리2주유소,1557.0,1417.0,35.215329,128.944364
...,...,...,...,...,...,...,...
342,사상구,부산 사상구 낙동대로 1320,대림주유소,1679.0,1549.0,35.177934,128.976559
343,사상구,부산 사상구 낙동대로 935,광신홀딩스㈜ 직영 낙동대로주유소,1689.0,1544.0,35.144005,128.969270
344,사상구,부산 사상구 학감대로 73 (학장동),행복한셀프주유소,1689.0,1549.0,35.138868,128.991863
345,사상구,부산 사상구 낙동대로 1380,지에스칼텍스(주)서부주유소,1739.0,1599.0,35.183325,128.976685


In [40]:
import folium

m = folium.Map(location=[35.1796, 129.0756], zoom_start=12)

for _, row in c_data.iterrows():
    popup_html = f"""
    <div style="font-size:13px;">
        <b>{row['상호']}</b><br>
        휘발유: {int(row['휘발유'])}원
    </div>
    """
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=folium.Popup(popup_html, max_width=200),
        icon=folium.Icon(color='blue')
    ).add_to(m)

for _, row in e_data.iterrows():
    popup_html = f"""
    <div style="font-size:13px;">
        <b>{row['상호']}</b><br>
        휘발유: {int(row['휘발유'])}원
    </div>
    """
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=folium.Popup(popup_html, max_width=200),
        icon=folium.Icon(color='red')
    ).add_to(m)

m
